## Telco-Churn-Analysis

This project uses the Telco Customer Churn dataset from Kaggle, originally published by IBM Sample Data Sets. The dataset contains information on 7,043 telecom customers, including their demographics, account details, service usage, and whether they have churned. The goal is to analyze patterns in customer behavior, handle class imbalance, and build machine learning models to predict churn. We apply techniques like EDA, feature engineering, and ensemble methods to improve prediction accuracy. The final outcome includes business insights and retention strategy recommendations.

### 1. Set up the Environment

In [33]:
import os
import pandas as pd
import numpy as np

### 2. Basic Processing

In [34]:
df = pd.read_csv('../Data/Raw/Telco_Churn.csv')
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [35]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


### 3. Data Quality Check

In [36]:
# Data types
print("Data Types:")
print(df.dtypes)

# Duplicates
print("\nDuplicates:")
print(df.duplicated().sum())

# Missing values
print("\nMissing Values:")
print(df.isnull().sum())

print("\nUnique Values:")
for col in df.columns[:]:
    print(f"{col}: {df[col].unique()[:]}")

Data Types:
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

Duplicates:
0

Missing Values:
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
Pa

The dataset contains no missing values,no duplicates. however, there is a data type inconsistency in the 'TotalCharges' column. Although this column should represent numerical values, it is stored as a string (object) type. This likely indicates the presence of non-numeric characters or blank entries in some rows. Therefore, a separate analysis is required to identify and handle these anomalies before converting the column to a proper numeric format.

In [37]:
# Create an empty list to collect rows with anomalies
anomalies = []

# Loop through the DataFrame
for i in range(len(df)):
    value = df.loc[i, 'TotalCharges']
    try:
        float(value)  # Try converting to float
    except ValueError:
        anomalies.append(df.loc[i])  # Collect the whole row

# Convert the list to a new DataFrame
anomaly_df = pd.DataFrame(anomalies)
anomaly_df


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


All the relavant columns' `tenure` column has 0 value. For further inspection let's analyze the `tenure` column

In [38]:
new_customers = []

for i in range(len(df)):
    if df.loc[i, 'tenure']==0:
        new_customers.append(df.loc[i])
    
new_customers_df = pd.DataFrame(new_customers)
new_customers_df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


##### Handling Anomalies in `TotalCharges` for New Customers

During data inspection, it was observed that some rows in the `TotalCharges` column contain blank strings (`' '`) instead of numeric values. Upon further analysis, it was found that **all such rows correspond to customers with a `tenure` of 0**, meaning they likely just signed up and have **not yet completed a full billing cycle**.

However, these customers already have a value in the `MonthlyCharges` column, which represents the expected charge for a typical month based on their chosen plan.

To address this:

- A new binary column, `is_new_customer`, is created to indicate whether a customer is new (`tenure == 0`).
- The empty `TotalCharges` values are logically imputed with `0`, assuming that no charges have been accumulated yet.
- The `TotalCharges` column is then safely converted to numeric type for further analysis.

This approach preserves data integrity, avoids arbitrary imputation, and provides the model with a useful signal (`is_new_customer`) that may improve its understanding of customer behavior.


### 3.1 Create new feature `is_new_customer` and handle anomalies in `TotalCharges`

In [39]:
df['is_new_customer'] = df['tenure'].apply(lambda x: 1 if x == 0 else 0)
df['TotalCharges'] = df['TotalCharges'].replace(" ", "0").astype(float)

In [40]:
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,is_new_customer
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.50,No,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,0
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,0
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,0
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.40,No,0
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,Month-to-month,No,Mailed check,29.75,301.90,No,0
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,0
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No,0


### 4. Target Variable Analysis

In [45]:
churn_counts = df['Churn'].value_counts()
churn_percentages = df['Churn'].value_counts(normalize=True)*100

summary = pd.DataFrame({
                    'Count' : churn_counts,
                    'percentage' : churn_percentages
                    })

summary

,Count,percentage
Churn,,
No,5174,73.463013
Yes,1869,26.536987


#### Target Variable Analysis: Churn Distribution & Class Imbalance

The target variable in this dataset is `Churn`, which indicates whether a customer has left the telecom service (Yes) or not (No). Below is the class distribution:

- **No (Non-Churned Customers)**: ~73%
- **Yes (Churned Customers)**: ~27%

This distribution indicates a **moderate class imbalance**, where the majority of customers have not churned.

---

#### Implications of Class Imbalance

- **Bias in Predictive Models**: Machine learning models tend to favor the majority class, which in this case is "No". This can result in deceptively high accuracy scores while performing poorly at identifying actual churn cases.
  
- **Reduced Sensitivity**: The model may have low recall for the minority class ("Yes"), which is critical for churn prediction since our goal is to **identify customers likely to leave**.

- **Business Risk**: Misclassifying churners as non-churners means missed opportunities for customer retention strategies, potentially leading to revenue loss.

- **Metric Selection**: Due to the imbalance, we should avoid relying on overall accuracy. Instead, evaluation metrics like:
  - **Precision**
  - **Recall**
  - **F1-Score**
  - **ROC-AUC**
  ...should be prioritized to get a balanced view of performance.

---

To address this imbalance, we can explore:
- **Resampling techniques** (e.g., SMOTE, RandomOversampling, Undersampling).
- **Cost-sensitive learning** (e.g., class weights in models).
- **Threshold tuning** to optimize recall/precision trade-offs for churn prediction.

The presence of class imbalance must be kept in mind throughout model development to ensure the results are both **statistically valid and practically useful** in a business context.


### 5. Feature Overview

In [50]:
demographic_features = [
                        "gender", "SeniorCitizen", "Partner", "Dependents"
                        ]

behavioral_features = [
                        "tenure", "PhoneService", "MultipleLines", "InternetService",
                        "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
                        "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "is_new_customer"
                        ]

financial_features = [
                    "MonthlyCharges", "TotalCharges", "PaymentMethod"
                    ]   




To support a clean and modular preprocessing pipeline, features have been categorized into three main groups:

- **Demographic Features:** Represent static characteristics of the customer.
- **Behavioral Features:** Capture how the customer uses the services.
- **Financial Features:** Reflect the customer’s spending and billing information.

This categorization enables more efficient preprocessing and better model interpretability.


In [49]:
df.to_csv('../Data/Processed/Telco_Churn_Data_Quality_Issues_Handled.csv')